In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import vectorbt as vbt

from dffc._holt_winters import HW

d:\anaconda3\envs\quant\Lib\site-packages\telegram\utils\request.py:54: UserWarning:

python-telegram-bot wasn't properly installed. Please refer to README.rst on how to properly install.



In [3]:
from dffc.fund_data import register_fund_data

register_fund_data()

FundData already exists in vectorbt namespace


In [4]:
ndgd_data = vbt.FundData.download(
    ['007467', '110022'], 
    names = ['ND', 'GD'],
    start='2025-01-01',
    end="2025-08-01"
)

In [4]:
ndgd_data = ndgd_data.update()

In [4]:
ndgd_data.get('unit_value')

symbol,007467,110022
date,,
2025-01-02 00:00:00+08:00,1.6138,3.526
2025-01-03 00:00:00+08:00,1.5974,3.496
2025-01-06 00:00:00+08:00,1.6011,3.466
2025-01-07 00:00:00+08:00,1.6043,3.486
2025-01-08 00:00:00+08:00,1.6114,3.505
...,...,...
2025-07-28 00:00:00+08:00,1.6773,3.417
2025-07-29 00:00:00+08:00,1.6682,3.399
2025-07-30 00:00:00+08:00,1.6749,3.420


In [18]:
hw = HW.run(ndgd_data.get('unit_value')['007467'], alpha=0.2, beta=0.1, gamma=0.1, m=5)

1.6056000000000001 -0.008944000000000018 [0.99499956 0.99073483 0.99303175 1.00625013 1.01498374]
